In [ ]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from datetime import date
import math
from scipy.stats import norm

from pandas_datareader import data as pdr
from scipy.stats import norm, t
import seaborn as sns
import statsmodels.api as sm
from dateutil.parser import parse
import scipy.optimize as sc

from py_vollib.black_scholes import black_scholes as bs
from py_vollib.black_scholes.greeks.analytical import vega

<font size="5">**Question 1**

In [ ]:
current_date = date(2022,3,13)
last_date = date(2022,4,15)
T = ((last_date - current_date).days) / 255

S = 165
K = 165
r = 0.0025
sigma = 0.2
dividend_rate = 0.0053
b = r - dividend_rate

In [ ]:
def greek_delta(S, K, T, sigma, r, b, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    sigma: Implied Volatility
    r: Risk free rate
    b: cost of carry
    '''
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    if optionType == "Call" or optionType == "call":
        delta = norm.cdf(d1,0,1)
    elif optionType == "Put" or "put":
        delta = -norm.cdf(-d1,0,1)
    else:
        print("Wrong option type")
        return 0
    
    return delta

In [ ]:
def greek_gamma(S, K, T, sigma, r, b, optionType):
    
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    gamma = norm.pdf(d1, 0, 1) / (S*sigma*np.sqrt(T))
        
    return gamma

In [ ]:
def greek_vega(S, K, T, sigma, r, b, optionType):
    
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    vega = S*norm.pdf(d1, 0, 1)*np.sqrt(T)
        
    return vega*0.01

In [ ]:
def greek_theta(S, K, T, sigma, r, b, optionType):
    
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if optionType == "Call" or optionType == "call":
        theta = -S * norm.pdf(d1,0,1) * sigma / (2*np.sqrt(T)) - r * K * np.exp(-r*T) * norm.cdf(d2,0,1)
    elif optionType == "Put" or "put":
        theta = -S * norm.pdf(d1,0,1) * sigma / (2*np.sqrt(T)) + r * K * np.exp(-r*T) * norm.cdf(-d2,0,1)
    else:
        print("Wrong option type")
        return 0
    
    return theta/365

In [ ]:
def greek_rho (S, K, T, sigma, r, b, optionType):
    
    d1 = (np.log(S/K) + (b + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if optionType == "Call" or optionType == "call":
        rho = K*T*np.exp(-r*T)*norm.cdf(d2,0,1)
    elif optionType == "Put" or "put":
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2,0,1)
    else:
        print("Wrong option type")
        return 0

    return rho*0.01

In [ ]:
call_delta = greek_delta(S, K, T, sigma, r, b, optionType = "Call")
put_delta = greek_delta(S, K, T, sigma, r, b, optionType = "Put")

call_gamma = greek_gamma(S, K, T, sigma, r, b, optionType = "Call")
put_gamma = greek_gamma(S, K, T, sigma, r, b, optionType = "Put")

call_vega = greek_vega(S, K, T, sigma, r, b, optionType = "Call")
put_vega = greek_vega(S, K, T, sigma, r, b, optionType = "Put")

call_theta = greek_theta(S, K, T, sigma, r, b, optionType = "Call")
put_theta = greek_theta(S, K, T, sigma, r, b, optionType = "Put")

call_rho = greek_rho(S, K, T, sigma, r, b, optionType = "Call")
put_rho = greek_rho(S, K, T, sigma, r, b, optionType = "Put")

name = ["Delta", "Gamma", "Vega", "Theta", "Rho"]
call_greeks = [call_delta, call_gamma, call_vega, call_theta, call_rho]
put_greeks = [put_delta, put_gamma, put_vega, put_theta, put_rho]

result = pd.DataFrame(list(zip(name, call_greeks, put_greeks)), columns = ["Option Greeks", "Call", "Put"])
result

In [ ]:
def american_without_div (K, startDate, endDate, S0, sigma, r, N, optionType):
    '''
    S0: Underlying Price
    K: Strike Price
    T: Time to Maturity by trading days in a year
    sigma: Implied Volatility
    r: Risk free rate
    N: number of time steps
    u: up-factor in binomial tree
    d: down-factor in binomial tree
    p: probability for price up
    disc: discount rate
    *format of Date: startDate = date(2022, 2, 25)
    '''
    
    days = (endDate - startDate).days
    tradingDays = 255
    T = days / tradingDays #using trading days 
    t = T/N
    disc = np.exp(-r*t)
    u = np.exp(sigma*np.sqrt(t))
    d = 1/u
    p = (np.exp(r*t) - d) / (u-d)
    
    #calculate the stock price at maturity
    S = np.zeros(N+1) #there are N+1 nodes at maturity (last column)
    for j in range(0, N+1):
        S[j] = S0 * u**j * d**(N-j)
        
    #calculate option payoff
    C = np.zeros(N+1)
    for j in range (0, N+1):
        if optionType == "Call" or optionType == "call":
            C[j] = max(0, S[j] - K)
        elif optionType == "Put" or "put":
            C[j] = max(0, K - S[j])
        else:
            print("Wrong option type")
            return 0
            
    #calculate the option value
    for i in np.arange(N-1,-1,-1):
        for j in range(0, i+1):
            S = S0 * u**j * d**(i-j)
            C[j] = disc * ( p*C[j+1] + (1-p)*C[j] )
            if optionType == "Call" or optionType == "call":
                C[j] = max(C[j], S - K)
            elif optionType == "Put" or "put":
                C[j] = max(C[j], K - S)
            else:
                print("Wrong option type")
                return 0
                
    return C[0]

In [ ]:
def american_with_div (K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType):
    '''
    S0: Underlying Price
    K: Strike Price
    sigma: Implied Volatility
    r: Risk free rate
    u: up-factor in binomial tree
    d: down-factor in binomial tree
    p: probability for price up
    div: dividend value
    *format of Date: startDate = date(2022, 2, 25)
    '''
    
    tradingDays = 255
    #calculate days from start to div, start to end, div to end
    start2div = (divDate - startDate).days
    div2end = (endDate - divDate).days
    start2end = (endDate - startDate).days
    
    #calculate the rate of each part using trading days 
    ttm = start2end / tradingDays
    ttm_start2div = start2div / tradingDays
    ttm_div2end = div2end / tradingDays
    
    #calculate u, d, and p
    u = np.exp(sigma*np.sqrt(ttm))
    d = 1/u
    p = (np.exp(r*ttm) - d) / (u-d)
    
    #calculate the stock price and option price at maturity
    D0 = div / (1+r)**ttm_start2div
    S0 = S0 - D0
    
    S = np.zeros(N+1)
    for j in range(0, N+1):
        S[j] = S0 * u**(N-j) * d**j
        
    C = np.zeros(N+1)
    for j in range (0, N+1):
        if optionType == "Call" or optionType == "call":
            C[j] = max(0, S[j] - K)
        elif optionType == "Put" or "put":
            C[j] = max(0, K - S[j])
        else:
            print("Wrong option type")
            return 0
            
    #calculate the stock price at dividend period
    S1 = np.zeros(N)
    for j in range(0, N):
        S1[j] = S0 * u**(N-1-j) * d**j
    
    #calculate the option price executing at dividend period
    C1_exe = np.zeros(N)
    for j in range (0, N):
        if optionType == "Call" or optionType == "call":
            temp = max(0, S1[j] - K)
            if temp > 0:
                C1_exe[j] = temp + div
            else:
                C1_exe[j] = temp
        elif optionType == "Put" or "put":
            temp = max(0, K - S1[j])
            if temp > 0:
                C1_exe[j] = temp + div
            else:
                C1_exe[j] = temp
        else:
            print("Wrong option type")
            return 0
            
    #calculate the option price executing at maturity at dividend period
    C1 = np.zeros(N)
    for j in range(0, N):
        C1[j] = (p*C[j] + (1-p)*C[j+1]) / (1+r)**ttm_div2end
        
    #compare C1 and C1_exe to determined whether execute option at dividend period
    C1_max = np.zeros(N)
    for j in range(N):
        C1_max[j] = max(C1[j], C1_exe[j])
        
    #calculate the option value
    C0 = (p*C1_max[0] + (1-p)*C1_max[1]) / (1+r)**ttm_start2div
    
    return C0

In [ ]:
#without_div
K = 165
startDate = date(2022, 3, 13)
endDate = date(2022, 4, 15)
S0 = 165
sigma = 0.2
r = 0.0025
dividend_rate = 0.0053
b = r - dividend_rate
N = 1

american_without_call = american_without_div(K, startDate, endDate, S0, sigma, r, N, optionType = "Call")
american_without_put = american_without_div(K, startDate, endDate, S0, sigma, r, N, optionType = "Put")

#with_div
K = 165
startDate = date(2022, 3, 13)
endDate = date(2022, 4, 15)
divDate = date(2022, 4, 11)
div = 0.88
S0 = 165
sigma = 0.2
r = 0.0025
couponRate = 0.0053
b = r - couponRate
N = 2

american_withDiv_call = american_with_div(K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType = "Call")
american_withDiv_put = american_with_div(K, startDate, endDate, divDate, div, S0, sigma, r, N, optionType = "Put")

name = ["American option value without dividends", "American option value with dividends"]
call_value = [american_without_call, american_withDiv_call]
put_value = [american_without_put, american_withDiv_put]

result = pd.DataFrame(list(zip(name, call_value, put_value)), columns = ["American option price with/without discrete dividends", "Call Option Value", "Put Option Value"])
result

<font size="5">**Question 2**

In [ ]:
port = pd.read_csv("problem2.csv")
ret = pd.read_csv("DailyReturn.csv", index_col = 0)['AAPL']

In [ ]:
def impliedVolatility (S, X, T, r, market_price, tol, optionType):
    '''
    S: Underlying Price
    X: Strike Price
    T: Time to Maturity
    r: Risk free rate
    tol: tolerance
    '''
    
    max_iter = 9999
    iv_old = 0.5
    
    for i in range(max_iter):
        bs_price = bs(optionType, S, X, T, r, iv_old)
        vega_price = vega(optionType, S, X, T, r, iv_old) * 100
        gap = bs_price - market_price
        
        iv_new = iv_old - gap/vega_price
        new_bs_price = bs(optionType, S, X, T, r, iv_new)
        
        if (abs(iv_old - iv_new) < tol or abs(new_bs_price - bs_price) < tol):
            break
        
        iv_old = iv_new
        
    impliedVolatility = iv_new
    return impliedVolatility

In [ ]:
def AmericanValue(port, underlying_values, simulated_date):
    result = pd.DataFrame(port['Portfolio'])
    
    for price in underlying_values:
        values = []
        for i in range(len(port)):
            if port['Type'][i] == 'Option':
                if port['OptionType'][i] == 'Call': #value = american_with_div (K = port["Strike"][i], startDate, endDate, divDate, div, S0 = price, sigma = port["ImpliedVolatility"][i], r, N, optionType = "Call")
                    value = american_with_div ( port["Strike"][i], startDate, endDate, divDate, div, price, port["ImpliedVolatility"][i], r, N, "Call")
                else:
                    value = american_with_div (port["Strike"][i], startDate, endDate, divDate, div, price, port["ImpliedVolatility"][i], r, N, "Put")
                values.append(value)
            else:
                values.append(price)
        result[str(price)] = np.array(values)
        
    result = result.groupby('Portfolio').sum()
    return result

In [ ]:
def parametricVaR (mean, std, distribution, alpha, dof):
    if distribution == "normal":
        VaR = (1-alpha/100) * std - mean #since normal distribution is symmetric, it is easy to calculate by using 1-alpha/100
    elif distribution == "t-distribution":
        VaR = np.sqrt((dof-2)/dof) * t.ppf(1-alpha/100, dof) * std - mean
    else:
        raise TypeError("Expected the distribution to be normal or t-distribution.")
    return VaR

In [ ]:
def parametricCVaR (mean, std, distribution, alpha, dof):
    if distribution == "normal":
        CVaR = (alpha/100)**-1 * norm.pdf( norm.ppf(alpha/100) ) * std - mean
    elif distribution == "t-distribution":
        x = t.ppf(alpha/100, dof)
        CVaR = -1/(alpha/100) * (1-dof)**-1 * (dof-2+x**2) * t.pdf(x,dof) *std - mean
    else:
        raise TypeError("Expected the distribution to be normal or t-distribution.")
    return CVaR

In [ ]:
startDate = date(2022, 2, 25)
endDate = date(2022, 3, 18)
T = (endDate - startDate).days / 255

S = 164.85
r = 0.0025
dividend_rate = 0.0053
b = r - dividend_rate
tol = 1e-5

In [ ]:
IV = []

for i in range(len(port.index)):
    if port.Type[i] == "Stock":
        IV.append(None)
    else:
        if port.OptionType[i] == "Call":
            iv = impliedVolatility(S, port.Strike[i], T, r, port.CurrentPrice[i], tol, "c")
            IV.append(iv)
        else:
            iv = impliedVolatility(S, port.Strike[i], T, r, port.CurrentPrice[i], tol, "p")
            IV.append(iv)
    
port["ImpliedVolatility"] = IV
port

In [ ]:
#Simulation
startDate = date(2022, 2, 25)
endDate = date(2022, 3, 18)
divDate = date(2022, 3, 15)
div = 1
r = 0.0025
N=2

In [ ]:
port['Value'] = port["CurrentPrice"]*port['Holding']
portval = port.groupby('Portfolio').sum()['Value']

In [ ]:
std = ret.std()
nsample = 5000
rets_sim = scipy.stats.norm(0, std).rvs((10, nsample))
price_sim = pd.DataFrame(164.85 * (1 + rets_sim).prod(axis=0))

portval_bar = AmericanValue(port, price_sim.values,date(2022,3,7))
port_list = (portval_bar.div(portval,axis=0)-1).T

In [ ]:
#Calculate VaR and CVaR
alpha = 5
dof = nsample - 1
num_days = (endDate - startDate).days

name = []
Mean, VaR, CVaR, VaR_val, CVaR_val = [], [], [], [], []
for strategy in port_list.columns:
    data = port_list[strategy]
    
    Mean.append(data.mean())
    
    empVaR = np.percentile(data, alpha)
    belowVaR = data <= empVaR
    empCVaR = data[belowVaR].mean()
    
    empVaR_val = port_list[strategy].sum() * empVaR * np.sqrt(num_days)
    empCVaR_val = port_list[strategy].sum() * empCVaR * np.sqrt(num_days)
    
    VaR.append(empVaR)
    CVaR.append(empCVaR)
    VaR_val.append(empVaR_val)
    CVaR_val.append(empCVaR_val)
    
    name.append(strategy)

result = pd.DataFrame(list(zip(name, Mean, VaR, CVaR, VaR_val, CVaR_val)), columns = ["Strategy", "Mean", "Empirical VaR", "Empirical CVaR", "Empirical VaR Value ($)", "Empirical CVaR Value ($)"])
result

In [ ]:
#Delta Normal
alpha = 5
dof = nsample - 1
distribution = "normal"

name = []
Mean, deltaVaR, deltaCVaR, deltaVaR_val, deltaCVaR_val = [], [], [], [], []
for strategy in port_list.columns:
    data = port_list[strategy]
    
    Mean.append(data.mean())
    
    delta_VaR = parametricVaR (data.mean(), data.std(), distribution, alpha, dof)
    delta_CVaR = parametricCVaR (data.mean(), data.std(), distribution, alpha, dof)
    
    delta_VaR_val = port_list[strategy].sum() * delta_VaR * np.sqrt(num_days)
    delta_CVaR_val = port_list[strategy].sum() * delta_CVaR * np.sqrt(num_days)
    
    deltaVaR.append(delta_VaR)
    deltaCVaR.append(delta_CVaR)
    deltaVaR_val.append(delta_VaR_val)
    deltaCVaR_val.append(delta_CVaR_val)
    
    name.append(strategy)

result = pd.DataFrame(list(zip(name, Mean, deltaVaR, deltaCVaR, deltaVaR_val, deltaCVaR_val)), columns = ["Strategy", "Mean", "Delta VaR", "Delta CVaR", "Delta VaR Value ($)", "Delta CVaR Value ($)"])
result

<font size="5">**Question 3**

In [ ]:
ret = pd.read_csv("DailyReturn.csv")
ff = pd.read_csv("F-F_Research_Data_Factors_daily.csv")
mom = pd.read_csv("F-F_Momentum_Factor_daily.csv")

In [ ]:
#merge the ff and mom
data = pd.merge(ff, mom)

#transfer percentage to value for all factors
data[["Mkt-RF", "SMB", "HML", "RF", "Mom"]] = data[["Mkt-RF", "SMB", "HML", "RF", "Mom"]] / 100

data


In [ ]:
def Date_to_int64 (vector):
    n = len(vector)
    dateInt = np.zeros(n)
    
    for i in range(n):
        date = vector[i]
        data_list = date.split("/")
        
        temp = []
        temp.append(data_list[2])
        temp.append(data_list[0])
        temp.append(data_list[1])
        
        dateInt[i] = int("".join(temp))
        
    return dateInt

In [ ]:
ret["Date"] = Date_to_int64(ret["Date"])

In [ ]:
data_ret = pd.merge(data, ret)

In [ ]:
'''
The alpha(intercept) is 0 for this function
First row: parameter value of beta_market for each stock
Second row: parameter value of beta_SMB for each stock
Third row: parameter value of beta_HML for each stock
Fourth row: parameter value of beta_UMD for each stock
'''

def famafrenchParameter(dataframe, stocks):
    m = len(stocks)
    n = len(dataframe)
    
    beta_market = np.zeros(m)
    beta_SMB = np.zeros(m)
    beta_HML = np.zeros(m)
    beta_UMD = np.zeros(m)
    
    for i in range(m):
        y = data_ret[ stocks[i] ] - data_ret["RF"]
        X = data_ret[ ["Mkt-RF", "SMB", "HML", "Mom"] ]
            
        model = sm.OLS(y,X)
        result = model.fit()
            
        beta_market[i] = result.params[0]
        beta_SMB[i] = result.params[1]
        beta_HML[i] = result.params[2]
        beta_UMD[i] = result.params[3]
    
    data = [beta_market, beta_SMB, beta_HML, beta_UMD]
    paramData = pd.DataFrame(data, columns = stocks)
    return paramData

In [ ]:
stocks = ["AAPL", "FB", "UNH", "MA", "MSFT", "NVDA", "HD", "PFE", "AMZN", "BRK-B", "PG", "XOM", "TSLA", "JPM", "V", "DIS", "GOOGL", "JNJ", "BAC", "CSCO"]
paramData = famafrenchParameter(data_ret, stocks)
paramData

In [ ]:
#dailyn return
def annualReturn_4factors (factorData, paramData, stocks):
    n = len(factorData)
    m = len(stocks)
    
    for i in range(m):
        stock = stocks[i]
        factorData[stock] = None
        for j in range(n):
            factorData[stock][j] = factorData["RF"][j] + paramData[stock][0]*factorData["Mkt-RF"][j] + paramData[stock][1]*factorData["SMB"][j] + paramData[stock][2]*factorData["HML"][j] + paramData[stock][3]*factorData["Mom"][j]
            
    return factorData

In [ ]:
data_dailyReturn = annualReturn_4factors(data, paramData, stocks)

In [ ]:
#annual return
data_dailyReturn["Year"] = None
for i in range( len(data_dailyReturn) ):
    dateformat = parse(str(data_dailyReturn["Date"][i]))
    data_dailyReturn["Year"][i] = dateformat.year
    
data_dailyReturn

In [ ]:
year_list = np.arange(1929, 2023, 1)
year_num = len(year_list)
n = len(data_dailyReturn)
m = len(stocks)

annualReturn = np.zeros((year_num, m))

for i in range(0, year_num):
    year = year_list[i]
    temp = data_dailyReturn[ data_dailyReturn["Year"] == year ]
    temp_len = len(temp)
    for j in range(m):
        stock = stocks[j]
        sqrtGeo = 1
        for k in range(0, temp_len):
            sqrtGeo = sqrtGeo * (1 + temp[stock].iloc[k])
        annualReturn[i][j] = sqrtGeo**(1/temp_len) - 1

annRetdf = pd.DataFrame(data = annualReturn, columns = stocks)
annRetdf #from 1926 to 2022

In [ ]:
covMatrix = annRetdf.cov()
covMatrix

In [ ]:
#set weight for each stocks
weights = np.ones(20)*(1/20)

#initial some important value
meanReturns = annRetdf.mean()

In [ ]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))*np.sqrt(252)
    return returns, std

In [ ]:
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

In [ ]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd

In [ ]:
def maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet=(0,1)):
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

In [ ]:
riskFreeRate = 0.0025
result = maxSR(meanReturns, covMatrix, riskFreeRate, constraintSet=(0,1))
maxSR, maxWeights = result['fun'], result['x']

In [ ]:
print("The maximum sharp ratio is: %f" % maxSR)
print("The weights for each stock should be:")
print(maxWeights)